In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from imgaug import augmenters as iaa
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, lr_scheduler
from torchsummary import summary


folder = r'C:\Users\Administrator\Documents\Computer-Vision\fmnist'

train_fmnist = datasets.FashionMNIST(folder, download = True, train=True)
validation_fmnist = datasets.FashionMNIST(folder, download=True, train=False)

image_train, label_train = train_fmnist.data, train_fmnist.targets
image_validation, label_validation = validation_fmnist.data, validation_fmnist.targets

class Data(Dataset):
    def __init__(self, x, y):
        super().__init__()
        x = x.float() / 255
        x = x.view(-1, 1, 28, 28)
        self.x, self.y = x, y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]

def DataLoaders():
    train_dl = DataLoader(Data(image_train, label_train), batch_size = 64, shuffle = True)
    validation_dl = DataLoader(Data(image_validation, label_validation), batch_size = 64, shuffle = True)
    return train_dl, validation_dl

train_dl, validation_dl = DataLoaders()

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size = 3)
        self.maxp1 = nn.MaxPool2d(2)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 128, kernel_size = 3)
        self.maxp2 = nn.MaxPool2d(2)
        self.relu2 = nn.ReLU()
        self.flat1 = nn.Flatten()
        self.line1 = nn.Linear(3200, 256)
        self.relu3 = nn.ReLU()
        self.line2 = nn.Linear(256, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxp1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.maxp2(x)
        x = self.relu2(x)
        x = self.flat1(x)
        x = self.line1(x)
        x = self.relu3(x)
        x = self.line2(x)
        return x

def compile():
    model = Model()
    loss_function = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr = 0.001)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,
                                               factor = 0.5,
                                               patience = 2,
                                               threshold = 0.001,
                                               verbose = True,
                                               min_lr = 0.01,
                                               threshold_mode = 'abs')
    
    return model, loss_function, optimizer, scheduler

model, loss_function, optimizer, scheduler = compile()

def for_training(x, y, model, loss_function, optimizer):
    model.train()
    optimizer.zero_grad()
    predicted = model(x)
    loss = loss_function(predicted, y)
    loss.backward()
    optimizer.step()
    accuracy = (torch.argmax(predicted, dim = 1) == y).float().mean().item()
    return loss.item(), accuracy

def for_validation(x, y, model, validation_dl, loss_function):
    model.eval()
    val_loss, val_accuracy = [], []
    for x, y, in validation_dl:
        prediction = model(x)
        loss = loss_function(prediction, y)
        acc = (torch.argmax(prediction, dim = 1) == y).float().mean().item()
        val_loss.append(loss.item())
        val_accuracy.append(acc)
        return np.mean(val_accuracy), np.mean(val_loss)


epochs = 10
train_loss, train_accuracy = [], []
for epoch in range(10):
    epoch_loss, epoch_accuracy = [], []
    for x, y in train_dl:
        loss, accuracy = for_training(x, y, model, loss_function, optimizer)
        epoch_loss.append(loss)
        epoch_accuracy.append(accuracy)
    train_loss.append(epoch_loss)
    train_accuracy.append(epoch_loss)
    mean_epoch_loss = np.mean(epoch_loss)
    mean_epoch_accuracy = np.mean(epoch_accuracy)
    print(f'epoch {epoch+1}/{epochs} \Train accuracy: {mean_epoch_accuracy} Train_loss: {mean_epoch_loss}')

    with torch.no_grad():
        mean_val_accuracy, mean_val_loss = for_validation(x, y, model, validation_dl, loss_function)
        print(f"Validation accuracy: {mean_val_accuracy} Validation loss: {mean_val_loss}\n")
    scheduler.step(mean_val_loss)

In [ ]:
from torchsummary import summary

summary(model, (1, 28, 28))

In [ ]:
image = 232
img = image_train[image]/255
img = img.view(28, 28)
img2 = np.roll(img, 1, axis = 1)
img3 = torch.Tensor(img2).unsqueeze(0).unsqueeze(0)
predict = model(img3).detach().numpy()
plt.title(validation_fmnist.classes[np.argmax(predict)])
plt.imshow(img, cmap = 'gray')
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Data transformations including augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load CIFAR-10 datasets
train_data = datasets.CIFAR10(root=folder, train=True, download=True, transform=transform_train)
test_data = datasets.CIFAR10(root=folder, train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_data, batch_size=100, shuffle=False, num_workers=2)

# Define CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 16x16

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 8x8

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 4x4
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(256*4*4, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.classifier(x)
        return x

# Model, loss, optimizer, and scheduler
model = CNN().cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=2, verbose=True)

# Training function
def train(model, train_loader, criterion, optimizer):
    model.train()
    total_loss, correct = 0, 0
    for data, target in train_loader:
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct += (output.argmax(dim=1) == target).sum().item()
    return total_loss / len(train_loader), correct / len(train_loader.dataset)

# Testing function
def test(model, test_loader, criterion):
    model.eval()
    total_loss, correct = 0, 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item()
            correct += (output.argmax(dim=1) == target).sum().item()
    return total_loss / len(test_loader), correct / len(test_loader.dataset)

# Train and evaluate the model
num_epochs = 50
best_acc = 0
early_stop_counter = 0
early_stop_patience = 5

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    test_loss, test_acc = test(model, test_loader, criterion)
    scheduler.step(test_loss)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
          f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')

    # Early stopping
    if test_acc > best_acc:
        best_acc = test_acc
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= early_stop_patience:
            print("Early stopping triggered.")
            break


<h1>Object Detection</h1>

<h1>Region-based Convolutional Neural Network

In [ ]:
# Using the felzenszwalb for simple Image segmentation
from torch_snippets import *
from skimage.segmentation import felzenszwalb

img = read('Hemanvi.jpeg', 1)
segmented = felzenszwalb(img, scale=200, min_size=100)
subplots(
    [img, segmented],
    titles = ['Original Image', 'Segmented Image'],
    sz = 5,
    nc=2
)

In [ ]:
from torch_snippets import *
import selectivesearch
import torch

img = read('Hemanvi.jpeg', 1)

def region_proposal(img):
    img_lbl, regions = selectivesearch.selective_search(img, scale = 200, min_size = 100)
    image_area = np.prod(img.shape[:2])
    candidates = []
    for r in regions:
        if r['rect'] in candidates: continue
        if r['size'] < (0.05 * image_area): continue
        if r['size'] > (1 * image_area) : continue
        x, y, w, h = r['rect']
        candidates.append(list(r['rect']))

    return candidates

x = region_proposal(img)

In [ ]:
def IoU(boxa, boxb, epsilon = 1e-5):
    x1 = max(boxa[0], boxb[0])
    y1 = max(boxa[1], boxb[1])
    x2 = min(boxa[2], boxb[2])
    y2 = min(boxa[3], boxb[3])
    width = x2- x1
    height = y2- y1
    if (width<0) or (height<0): return 0.0
    area_overlap = width * height
    area_a = (boxa[2] - boxa[0]) * (boxa[3] - boxa[1])
    area_b = (boxb[2] - boxb[0]) * (boxb[3] - boxb[1])
    combined_area = area_a + area_a - area_overlap
    iou = area_overlap / (combined_area + epsilon)
    return iou

In [ ]:
from torch_snippets import *
import torch
import torch.nn as nn
from torch.optim import Adam, lr_scheduler
from torch.utils.data import Dataset, DataLoader, random_split, TensorDataset
from torchsummary import summary
from torchvision import transforms as T, datasets, models
from torchvision.ops import nms
from torch_snippets import *
import selectivesearch

image_root = r'/home/zkllmt/Documents/AI Section/Datasets/Object Detection/open-images-bus-trucks/images/images'
df_ = pd.read_csv(r'/home/zkllmt/Documents/AI Section/Datasets/Object Detection/open-images-bus-trucks/df.csv')

class Data(Dataset):
    def __init__(self, df, root = image_root, transforms = None):
        self.df = df
        self.root = image_root
        self.transforms = transforms
        self.unique_IDs = self.df['ImageID'].unique()
    
    def __len__(self): return len(self.unique_IDs)

    def __getitem__(self, index) -> any:
        img_id = self.unique_IDs[index]
        file_path = f'{self.root}/{img_id}.jpg'
        img = read(file_path, 1)
        if self.transforms:
            img = self.transforms(img)
        
        h, w = img.shape[:2]
        df = self.df.copy()
        df = df[df['ImageID'] == img_id]
        boxes = df['XMin,YMin,XMax,YMax'.split(',')].values
        boxes = (boxes * np.array([w,h,w,h])).astype(np.int16).tolist()
        labels = df['LabelName'].tolist()
        return img, boxes, labels, file_path
data = Data(df)
#img, boxes, labels, file_path = data[2]
#show(img, bbs = boxes, texts=labels, text_sz=10, title = labels)

def get_candidates(img):
    lbls, regions = selectivesearch.selective_search(img, scale = 200, min_size=100)
    candidates = []
    img_area = np.prod(img.shape[:2])
    for i in regions:
        if i['rect'] in candidates: continue
        if i['size'] < (0.05 * img_area): continue
        if i['size'] > (1 * img_area): continue
        x, y, w, h = i['rect']
        candidates.append(list(i['rect']))
    return candidates

def get_iou(boxa, boxb, epsilon = 1e-5):
    x1 = max(boxa[0], boxb[0])
    y1 = max(boxa[1], boxb[1])
    x2 = min(boxa[2], boxb[2])
    y2 = min(boxa[3], boxb[3])
    
    width = (x2 - x1)
    height = (y2 - y1)

    if (width <0) or (height <0):
        return 0.0
    
    area_overlap = width * height

    area_a = (boxa[2] - boxa[0]) * (boxa[3] - boxa[1])
    area_b = (boxb[2] - boxb[0]) * (boxb[3] - boxb[1])
    combined = area_a + area_b - area_overlap
    iou = area_overlap / (combined + epsilon)
    return iou

FPATHS, GTBBS, CLSS, DELTAS, IOUS, ROIS = [],[],[],[],[],[]
N = 500

In [ ]:
for ix, (img, boxes, labels, file_path) in enumerate(data):
    if ix == 500: break

    H, W, C = img.shape
    candidates = get_candidates(img)
    candidates = np.array([(x, y, x+w, y+h) for x,y,w,h in candidates])
    ious, deltas, clss, rois = [],[],[],[]
    ious = np.array([[get_iou(candidate, _bb_) for candidate in candidates] for _bb_ in boxes]).T
    for jx, candidate in enumerate(candidates):
        cx, cy, cX, cY = candidate
        candidate_ious = ious[jx]
        best_ious_at = np.argmax(candidate_ious)
        best_iou = candidate_ious[best_ious_at]
        best_bb = _x, _y, _X, _Y = boxes[best_ious_at]
        
        if best_iou > 0.3: clss.append(labels[best_ious_at])
        else: clss.append("background")

        delta = np.array([cx-_x, cy-_y, cX-_X, cY-_Y]) / np.array([W, H, W, H])
        deltas.append(delta)
        rois.append(candidate / np.array([W, H, W, H]))

    FPATHS.append(file_path)
    GTBBS.append(boxes)
    CLSS.append(clss)
    DELTAS.append(deltas)
    IOUS.append(ious)
    ROIS.append(rois)

targets = pd.DataFrame(flatten(CLSS), columns=['Label'])
label2target = {l:t for t, l in enumerate(targets['Label'].unique())}
target2label = {t:l for l, t in label2target.items()}

def decode(_y):
    _, preds = _y.max(-1)
    return preds

normalize= T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
def preprocess_img(img):
    img = torch.tensor(img).permute(2, 0, 1)
    img = normalize(img)
    return img.float()

In [ ]:
class RCNNDataset(Dataset):
    def __init__(self, fpaths, rois, labels, deltas, gtbbs):
        self.fpaths = fpaths
        self.labels = labels
        self.deltas = deltas
        self.rois = rois
        self.gtbbs = gtbbs
        
    def __len__(self): return len(self.fpaths)

    def __getitem__(self, ix):
        fpath = str(self.fpaths[ix])
        img = read(fpath, 1)
        H, W, C = img.shape
        sh = np.array([W, H, W, H])
        gtbbs = self.gtbbs[ix]
        deltas = self.deltas[ix]
        labels = self.labels[ix]
        rois = self.rois[ix]
        bbs = np.array((rois)*sh).astype(np.uint16)
        crops = [img[y:Y, x:X] for (x,y,X,Y) in bbs]
        return img,crops,bbs,labels,deltas,gtbbs,fpath

    def collate_fn(self, batch):
        input, rois, rixs, labels, deltas =[],[],[],[],[]
        for ix in range(len(batch)):
            image, crops, image_bbs, image_labels, image_deltas, image_gt_bbs, image_fpath = batch[ix]
            crops = [cv2.resize(crop, (224,224)) for crop in crops]
            crops = [preprocess_image(crop/255.)[None] for crop in crops]
            input.extend(crops)
            labels.extend([label2target[c] for c in image_labels])
            deltas.extend(image_deltas)
        input = torch.cat(input).to
        labels = torch.Tensor(labels).long()
        deltas = torch.Tensor(deltas).float()
        return input, labels, deltas


n_train = 9*len(FPATHS)//10
train_ds = RCNNDataset(FPATHS[:n_train], ROIS[:n_train], CLSS[:n_train], DELTAS[:n_train], GTBBS[:n_train])
test_ds = RCNNDataset(FPATHS[n_train:], ROIS[n_train:], CLSS[n_train:], DELTAS[n_train:], GTBBS[n_train:])
train_loader = DataLoader(train_ds, batch_size=2, collate_fn=train_ds.collate_fn, drop_last=True)
test_loader = DataLoader(test_ds, batch_size=2, collate_fn=test_ds.collate_fn, drop_last=True)



<h1>FASTER RCNN</h1>

In [ ]:
from torch_snippets import *
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD, lr_scheduler
from torchsummary import summary
from torchvision import models, datasets, transforms as transforms
from torchvision.models import vgg16
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
from torch.utils.tensorboard.writer import SummaryWriter
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, fasterrcnn_resnet50_fpn
import torchvision
from glob import glob   
from tqdm import tqdm


image_root = r'/home/zkllmt/Documents/AI Section/Datasets/Object Detection/open-images-bus-trucks/images/images'
df = pd.read_csv(r'/home/zkllmt/Documents/AI Section/Datasets/Object Detection/open-images-bus-trucks/df.csv')

label2target = {l:t+1 for t, l in enumerate(df['LabelName'].unique())}
label2target['background'] = 0

target2label = {t:l for l, t in label2target.items()}
background_class = label2target['background']
num_classes = len(label2target)

def preprocess_image(img): 
    img = torch.tensor(img).permute(2, 0, 1) 
    return img.float()

class Data(Dataset):
    w, h = 224, 224
    def __init__(self, df, image_root = image_root):
        self.df = df
        self.image_dir = image_root
        self.files = glob(os.path.join(self.image_dir, '*'))
        self.image_infos = self.df.ImageID.unique()
    
    def __getitem__(self, ix):
        image_id = self.image_infos[ix]
        image_path = find(image_id, self.files)
        img = Image.open(image_path).convert("RGB")
        img = np.array(img.resize((self.w, self.h), resample = Image.BILINEAR)) / 255.
        df = self.df.copy()
        data = df[df['ImageID'] == image_id]
        labels = data['LabelName'].values.tolist()
        data = data[['XMin','YMin','XMax','YMax']].values
        data[:, [0, 2]] *= self.w
        data[:, [1, 3]] *= self.h
        boxes = data.astype(np.uint32).tolist()
        target = {}
        target["boxes"] = torch.Tensor(boxes).float()
        target['labels'] = torch.Tensor([label2target[i] for i in labels]).long()
        img = preprocess_image(img)
        return img, target

    def collate_fn(self, batch):
        return tuple(zip(*batch))
    
    def __len__(self):
        return len(self.image_infos)


data = Data(df)

trn_ids, val_ids = train_test_split(df.ImageID.unique(), test_size=0.1, random_state=99)
trn_df, val_df = df[df['ImageID'].isin(trn_ids)], df[df['ImageID'].isin(val_ids)]
train_ds = Data(trn_df)
test_ds = Data(val_df)

train_loader = DataLoader(train_ds, batch_size=4, collate_fn=train_ds.collate_fn, drop_last=True)
test_loader = DataLoader(test_ds, batch_size=4, collate_fn=test_ds.collate_fn, drop_last=True)

def get_model():
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model


# Defining training and validation functions
def train_batch(inputs, model, optimizer):
    model.train()
    input, targets = inputs
    input = list(image for image in input)
    targets = [{k: v for k, v \
                in t.items()} for t in targets]
    optimizer.zero_grad()
    losses = model(input, targets)
    loss = sum(loss for loss in losses.values())
    loss.backward()
    optimizer.step()
    return loss, losses
@torch.no_grad()
def validate_batch(inputs, model):
    model.train()
#to obtain losses, model needs to be in train mode only
#Note that here we arn't defining the model's forward method
#hence need to work per the way the model class is defined
    input, targets = inputs
    input = list(image for image in input)
    targets = [{k: v for k, v in t.items()} for t in targets]
    optimizer.zero_grad()
    losses = model(input, targets)
    loss = sum(loss for loss in losses.values())
    return loss, losses

model = get_model()
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9,weight_decay=0.0005)
class Logger:
    def __init__(self, n_epochs):
        self.n_epochs = n_epochs
        self.epoch_logs = []

    def record(self, pos, **kwargs):
        # Display the logs as you progress
        print(f"Epoch {pos:.2f}: ", end="")
        for key, value in kwargs.items():
            print(f"{key}: {value:.4f} ", end="")
        print()  # Newline after each record

    def report_avgs(self, epoch):
        # Report average metrics at each epoch
        avg_logs = {key: sum([log[key] for log in self.epoch_logs]) / len(self.epoch_logs) 
                    for key in self.epoch_logs[0]}
        print(f"Avg metrics at epoch {epoch}: {avg_logs}")

    def plot_epochs(self, metrics):
        # Dummy function for plotting
        print(f"Plotting metrics: {metrics}")


n_epochs = 5
log = Logger(n_epochs)

for epoch in range(n_epochs):
    # Training loop
    _n = len(train_loader)
    for ix, inputs in enumerate(tqdm(train_loader, desc=f"Training epoch {epoch+1}")):
        loss, losses = train_batch(inputs, model, optimizer)
        loc_loss, regr_loss, loss_objectness, loss_rpn_box_reg = [losses[k] for k in ['loss_classifier', 'loss_box_reg', 'loss_objectness', 'loss_rpn_box_reg']]
        
        pos = (epoch + (ix + 1) / _n)
        log.record(pos, trn_loss=loss.item(), trn_loc_loss=loc_loss.item(), trn_regr_loss=regr_loss.item(),
                   trn_objectness_loss=loss_objectness.item(), trn_rpn_box_reg_loss=loss_rpn_box_reg.item())
    
    # Validation loop
    _n = len(test_loader)
    for ix, inputs in enumerate(tqdm(test_loader, desc=f"Validating epoch {epoch+1}")):
        loss, losses = validate_batch(inputs, model)
        loc_loss, regr_loss, loss_objectness, loss_rpn_box_reg = [losses[k] for k in ['loss_classifier', 'loss_box_reg', 'loss_objectness', 'loss_rpn_box_reg']]
        
        pos = (epoch + (ix + 1) / _n)
        log.record(pos, val_loss=loss.item(), val_loc_loss=loc_loss.item(), val_regr_loss=regr_loss.item(),
                   val_objectness_loss=loss_objectness.item(), val_rpn_box_reg_loss=loss_rpn_box_reg.item())

    if (epoch + 1) % (n_epochs // 5) == 0:
        log.report_avgs(epoch + 1)

log.plot_epochs(['trn_loss', 'val_loss'])

<h1>YOLO (You Only Look Once)</h1>

In [ ]:
from ultralytics import YOLO, checks, hub
checks()

hub.login('7872b0b7def37af38e72e525b78bef0c3c74e4facd')

model = YOLO('https://hub.ultralytics.com/models/VZ8ePp9uuGbBFLLbUD2h')
results = model.train()

<h1>SSD Object Detection</h1>

In [2]:
#385